In [64]:
#Copyright 2018, Seokjun Bu, Softcomputing LAB all rights reserved.
from keras import backend as K
from keras.utils import np_utils
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution3D, MaxPooling3D, AveragePooling3D
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import LSTM, TimeDistributed, CuDNNLSTM
from keras.layers import Input
from keras.models import Model
from keras.models import load_model
from keras.metrics import top_k_categorical_accuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import itertools
%matplotlib inline

In [65]:
# Handcrafted Function Def.
def wrapped_confusion_matrix(modelInput, feature, label, class_list=["first", "second"], normalize=False, fig_size=(5,5)):
    pred = modelInput.predict(feature)
    cnf_matrix = confusion_matrix(np.argmax(label, axis=1), np.argmax(pred, axis=1))
    np.set_printoptions(precision=2)
    plt.figure(figsize=fig_size) 
    plt.imshow(cnf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.xticks(np.arange(len(class_list)), class_list, rotation=45)
    plt.yticks(np.arange(len(class_list)), class_list)
    if(normalize):
        cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
    thresh = cnf_matrix.max() / 2.
    for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
        plt.text(j, i, cnf_matrix[i, j],
                 horizontalalignment='center',
                 color='white' if cnf_matrix[i, j] > thresh else 'black')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
def plot_history(model_input, mode='loss', val_mode=True, title='Entropy'):
    history = model_input.history
    if(val_mode):
        plt.plot(history.history[mode])
        plt.plot(history.history['val_'+mode])
        plt.title(title)
        plt.legend(['train', 'test'], loc='upper right')
        plt.show()
    else:
        plt.plot(history.history[mode])
        plt.title(title)
        plt.legend(['train'], loc='upper right')
        plt.show()

In [66]:
X_train = np.load('../dataset/test/X_train.npy')
Y_train = np.load('../dataset/test/Y_train.npy')
X_test = np.load('../dataset/test/X_test.npy')
Y_test = np.load('../dataset/test/Y_test.npy')
#print(X, Y)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)
print(Y_test)

(3000, 64, 64, 3) (3000, 3)
(1000, 64, 64, 3) (1000, 3)
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [67]:
X_train, X_test = X_train/255., X_test/255.

In [68]:
model_input = Input(shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))
model_hidden = Conv2D(16, kernel_size=(2, 2), padding='same', activation='relu')(model_input)
model_hidden = MaxPooling2D((2, 2))(model_hidden)
model_hidden = Conv2D(16, kernel_size=(2, 2), padding='same', activation='relu')(model_hidden)
model_hidden = MaxPooling2D((2, 2))(model_hidden)
model_hidden = Flatten()(model_hidden)
model_hidden = Dense(512, activation='relu')(model_hidden)
model_output = Dense(Y_train.shape[1], activation='softmax')(model_hidden)
model_cnn = Model(model_input, model_output)
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 64, 64, 16)        208       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 32, 32, 16)        1040      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               2097664   
__________

In [69]:
model_cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [70]:
model_cnn = model_cnn.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30, batch_size=200, verbose=2, shuffle=True)

Train on 3000 samples, validate on 1000 samples
Epoch 1/30
 - 19s - loss: 1.0054 - acc: 0.4850 - val_loss: 0.9441 - val_acc: 0.5170
Epoch 2/30
 - 13s - loss: 0.9643 - acc: 0.5070 - val_loss: 0.9427 - val_acc: 0.5170
Epoch 3/30
 - 14s - loss: 0.9614 - acc: 0.5070 - val_loss: 0.9440 - val_acc: 0.5170
Epoch 4/30
 - 13s - loss: 0.9577 - acc: 0.5070 - val_loss: 0.9426 - val_acc: 0.5170
Epoch 5/30
 - 14s - loss: 0.9580 - acc: 0.5070 - val_loss: 0.9428 - val_acc: 0.5170
Epoch 6/30
 - 13s - loss: 0.9585 - acc: 0.5070 - val_loss: 0.9486 - val_acc: 0.5170
Epoch 7/30
 - 14s - loss: 0.9585 - acc: 0.5070 - val_loss: 0.9451 - val_acc: 0.5170
Epoch 8/30
 - 13s - loss: 0.9588 - acc: 0.5070 - val_loss: 0.9498 - val_acc: 0.5170
Epoch 9/30
 - 14s - loss: 0.9610 - acc: 0.5070 - val_loss: 0.9479 - val_acc: 0.5170
Epoch 10/30
 - 14s - loss: 0.9570 - acc: 0.5070 - val_loss: 0.9414 - val_acc: 0.5170
Epoch 11/30
 - 14s - loss: 0.9549 - acc: 0.5070 - val_loss: 0.9408 - val_acc: 0.5170
Epoch 12/30
 - 13s - loss:

In [72]:
class_name_list_low = ['True', 'False']
wrapped_confusion_matrix(model_cnn, X_test, Y_test, class_list=class_name_list_low, normalize=False, fig_size=(5,5))

AttributeError: 'History' object has no attribute 'predict'